In [17]:
!pip uninstall --yes opencv-python-headless

Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Successfully uninstalled opencv-python-headless-4.5.5.64


In [21]:
!pip install opencv-python-headless==4.1.2.30

     |████████████████████████████████| 21.8 MB 24.1 MB/s 


In [39]:
import numpy as np
import pandas as pd
import cv2
import requests
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab.patches import cv2_imshow
pd.options.display.max_colwidth = 120
pd.options.display.max_rows = 120

## Read ADE metadata

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/fourthbrain/Week 12')

Mounted at /content/drive


In [4]:
dfh = pd.read_csv('ade20k_header.csv')
dfd = pd.read_csv('ade20k_detail.csv')
dfd15 = pd.read_csv('ade_coco_15pts.csv')

In [5]:
os.chdir('/content')

In [6]:
dfh.sample(5)

,header_id,width,height,filename,folder,scene
7122,19927,1024,768,ADE_train_00019927.jpg,ADE20K_2021_17_01/images/ADE/training/home_or_hotel/wet_bar,"['indoor', 'home or hotel', 'wet_bar']"
11274,5035,600,416,ADE_train_00005035.jpg,ADE20K_2021_17_01/images/ADE/training/shopping_and_dining/canteen,"['indoor', 'shopping and dining', 'canteen']"
9487,12039,2048,1351,ADE_train_00012039.jpg,ADE20K_2021_17_01/images/ADE/training/nature_landscape/mountain_path,"['outdoor', 'nature landscape', 'mountain_path']"
22173,17513,2048,1536,ADE_train_00017513.jpg,ADE20K_2021_17_01/images/ADE/training/urban/street,"['outdoor', 'urban', 'street']"
9063,9772,590,299,ADE_train_00009772.jpg,ADE20K_2021_17_01/images/ADE/training/nature_landscape/ice_floe,"['outdoor', 'nature landscape', 'ice_floe']"


In [7]:
dfd.sample(5)

,id,raw_name,depth_ordering_rank,crop,header_id,num_mask_points,xmin,ymin,xmax,ymax,width,height
213427,7,window,8,0,11393,44,1,490,1459,1086,1458,596
119517,63,right hand,62,0,8367,6,361,114,366,123,5,9
576835,63,left leg,62,0,18594,33,1467,1142,1497,1260,30,118
474628,67,leg,68,0,15291,12,434,904,477,977,43,73
107515,30,chair,31,1,7144,32,235,406,362,591,127,185


In [8]:
dfd15.sample(5)

,Unnamed: 0,id,raw_name,depth_ordering_rank,crop,header_id,num_mask_points,xmin,ymin,xmax,ymax,width,height,coco_label
44556,44556,0,car,1,0,17152,22,719,849,1680,1177,961,328,car
11562,11562,14,bed,15,0,20958,41,196,227,1280,960,1084,733,bed
16853,16853,5,chair,6,0,23768,44,337,414,441,571,104,157,chair
39030,39030,4,truck,5,0,9164,18,199,293,237,330,38,37,truck
35083,35083,1,truck,2,0,1071,18,1493,1015,1600,1115,107,100,truck


## Grab 300 random files from dfd15. Let's call this `df300`

In [9]:
sample = dfd15.header_id.sample(n=300, random_state=52).values
df300 = dfh.loc[dfh.header_id.isin(sample)]
df300.sample(5)

,header_id,width,height,filename,folder,scene
17946,5015,2200,1650,ADE_train_00005015.jpg,ADE20K_2021_17_01/images/ADE/training/urban/canal__urban,"['outdoor', 'urban', 'canal urban']"
21511,282,916,605,ADE_frame_00000282.jpg,ADE20K_2021_17_01/images/ADE/training/urban/street,"['outdoor', 'urban', 'street']"
14001,7754,1280,960,ADE_train_00007754.jpg,ADE20K_2021_17_01/images/ADE/training/transportation/ferryboat__cargo_deck,"['indoor', 'transportation', 'ferryboat cargo_deck']"
22911,18251,2048,1536,ADE_train_00018251.jpg,ADE20K_2021_17_01/images/ADE/training/urban/street,"['outdoor', 'urban', 'street']"
23301,18641,256,256,ADE_train_00018641.jpg,ADE20K_2021_17_01/images/ADE/training/urban/street,"['outdoor', 'urban', 'street']"


In [ ]:
%%bash
cp '/content/drive/My Drive/Colab Notebooks/fourthbrain/Week 12/df300.tar.gz' /content
cd /content
tar zxvf df300.tar.gz

## Prepare tf official implementation of maskrcnn

### Prepare environment and imports

In [12]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3336, done.
remote: Counting objects: 100% (3336/3336), done.
remote: Compressing objects: 100% (2771/2771), done.
remote: Total 3336 (delta 882), reused 1417 (delta 515), pack-reused 0
Receiving objects: 100% (3336/3336), 34.33 MiB | 32.31 MiB/s, done.
Resolving deltas: 100% (882/882), done.


In [13]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cd models/research
cp object_detection/packages/tf2/setup.py .
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1692480 sha256=70b4822a7f0bc2c981e00725d99c5aa7487395c661dc890624513358533b8776
  Stored in directory: /tmp/pip-ephem-wheel-cache-973tvrcf/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=3723810b035cd7976ef090c644b783a93b870aef083c944d9f75ec0be36486fd
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=7a416e3664575312d1a77aceb03de27c65af45c8bd8a6c74cd1235a61ba2f2bb
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=f80236a3856296878aac03ddb7772ac9abbb9ba701ff582010

bash: line 3: cd: models/research: No such file or directory
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which 

In [14]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageOps
from IPython.display import display

In [15]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

### Retrieve the model

In [16]:
model_name = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8' # mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28

In [17]:
!curl -O http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  462M  100  462M    0     0   140M      0  0:00:03  0:00:03 --:--:--  140M


In [18]:
!tar zxvf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/checkpoint
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/saved_model.pb
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/assets/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables/variables.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables/variables.index


## Prepare helpers

### Geometry

In [19]:
################################################################################
def mask2poly(mask, distance: int = 5):
    """
    Convert segmentation mask into an xy list of polygon points [x1, y1, x2, y2, ...].

    Parameters
    ----------
    mask : np array
        Object segmentation mask of an image.

    distance : int
        Euclidean distance between polygon points. We use numpy.linalg.norm, whose
        default is L2 norm = Euclidean distance.
    
    Returns
    -------
    list
        The xy list of polygon points [x1, y1, x2, y2, ...].
    """

    # Get contours of the mask. Should return 1 object in a common use case.
    contours, hier = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the contours' polygons.
    poly = []
    for obj in contours:
        # Set sentinel to np.inf to always pick up the first point.
        sentinel = [np.inf, np.inf]
        for point in obj:
            # Only keep the point if it's more than the specified distance.
            if np.linalg.norm(sentinel-point[0]) >= distance:
                poly.append(int(point[0][0]))
                poly.append(int(point[0][1]))
                sentinel = point[0]

    return poly

################################################################################
def get_crop_bbox(img, bbox, padding=None):
    """
    Return the cropped portion of the given image.

    Parameters
    ----------
    img : np array

    bbox : tuple
        Base bounding box of the crop (xmin, ymin, xmax, ymax).

    padding : tuple of ints, or float
        Padding around the base bounding box.
        If None, no padding is added to the box.
        If a tuple, specify (padleft, padtop, padright, padbottom).
        If a float, a uniform but random padding around the box will be calculated.
                First, we get a base padding of: min(bboxwidth, bboxheight) * padding_pct.
                Next, we add to/subtract with: padding * random.uniform(0, padding_pct).

    Returns
    -------
    np array, tuple
        The cropped portion of the image, and its bounding box coordinates (xmin, ymin, xmax, ymax).
    """

    # Get image shape, bbox coordinates and shape.
    height, width = img.shape[:2]
    xmin, ymin, xmax, ymax = bbox
    bboxwidth, bboxheight = xmax-xmin, ymax-ymin

    # Figure out padding in number-of-pixels.
    if padding == None:
        padleft, padtop, padright, padbottom = 0, 0, 0, 0

    else:
        if type(padding) == tuple:
            # Get requested padding.
            padleft, padtop, padright, padbottom = padding

        elif type(padding) == float:
            # Get base padding.
            padding_pct = padding
            padding = int( min(bboxwidth, bboxheight) * padding_pct )

            # Make random addition/subtraction from base padding.
            padleft, padtop, padright, padbottom = padding * (1 + np.random.uniform(low=-padding_pct, high=padding_pct, size=4))

        # Figure out extra room after requested padding.
        extraleft, extraright = xmin-padleft, width-xmax-padright
        extratop, extrabottom = ymin-padtop, height-ymax-padbottom

        # Center bbox if not enough room horizontally.
        padhorz = max(0, extraleft+extraright)//2
        padleft, padright = (padhorz, padhorz) if min(extraleft, extraright) < 0 else (padleft, padright)

        # Do the same vertically.
        padvert = max(0, extratop+extrabottom)//2
        padtop, padbottom = (padvert, padvert) if min(extratop, extrabottom) < 0 else (padtop, padbottom)

    # Calculate final bbox.
    xmin, xmax = int(xmin-padleft), int(xmax+padright)
    ymin, ymax = int(ymin-padtop), int(ymax+padbottom)

    return img[ymin:ymax, xmin:xmax], (xmin, ymin, xmax, ymax)

### Inference

In [20]:
################################################################################
def run_inference_for_single_image(model, image):
  # Convert to np array.
  image_np = np.array(image)

  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image_np)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]
  
  # Run inference.
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  return image_np, input_tensor, output_dict

################################################################################
def get_predictions(output_dict):
  num_detections = int(output_dict['num_detections'])
  predictions = {}
  predictions['num_detections'] = num_detections
  for k in ['boxes', 'classes', 'masks', 'scores']:
    key = 'detection_{}'.format(k)
    predictions[key] = output_dict[key][0, :num_detections].numpy()
  
  predictions['detection_classes'] = predictions['detection_classes'].astype(np.int64)
  
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
      predictions['detection_masks'],
      predictions['detection_boxes'],
      image_np.shape[0],
      image_np.shape[1]
    )
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
    predictions['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return predictions

## Try inference on some images from df300

In [ ]:
df300.head()

,header_id,filename,folder,imsize,scene
147,22687,ADE_train_00022687.jpg,ADE20K_2021_17_01/images/ADE/training/cultural/art_gallery,"[750, 1000, 3]","['indoor', 'cultural', 'art_gallery']"
198,1742,ADE_train_00001742.jpg,ADE20K_2021_17_01/images/ADE/training/cultural/art_studio,"[311, 478, 3]","['indoor', 'cultural', 'art_studio']"
359,2007,ADE_train_00002007.jpg,ADE20K_2021_17_01/images/ADE/training/cultural/auditorium,"[768, 1024, 3]","['indoor', 'cultural', 'auditorium']"
458,5391,ADE_train_00005391.jpg,ADE20K_2021_17_01/images/ADE/training/cultural/chapel,"[1200, 1600, 3]","['indoor', 'cultural', 'chapel']"
501,106,ADE_frame_00000106.jpg,ADE20K_2021_17_01/images/ADE/training/cultural/classroom,"[1467, 2200, 3]","['indoor', 'cultural', 'classroom']"


In [21]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Load the model.
model_dir = '{}/saved_model'.format(model_name)
model = tf.saved_model.load(str(model_dir))

In [22]:
df300.loc[df300.width > 900].sample(5)

,header_id,width,height,filename,folder,scene
2126,305,2200,1650,ADE_train_00000305.jpg,ADE20K_2021_17_01/images/ADE/training/home_or_hotel/bedroom,"['indoor', 'home or hotel', 'bedroom']"
22631,17971,2048,1536,ADE_train_00017971.jpg,ADE20K_2021_17_01/images/ADE/training/urban/street,"['outdoor', 'urban', 'street']"
22102,17442,2048,1536,ADE_train_00017442.jpg,ADE20K_2021_17_01/images/ADE/training/urban/street,"['outdoor', 'urban', 'street']"
22273,17613,2200,1650,ADE_train_00017613.jpg,ADE20K_2021_17_01/images/ADE/training/urban/street,"['outdoor', 'urban', 'street']"
2240,419,1024,768,ADE_train_00000419.jpg,ADE20K_2021_17_01/images/ADE/training/home_or_hotel/bedroom,"['indoor', 'home or hotel', 'bedroom']"


In [40]:
# Choose an image.
header_id = 17971
image_path = '/content/df300/' + df300.loc[df300.header_id==header_id].filename.values[0]
#image = Image.open(image_path)
image = mpimg.imread(image_path)

# Run inference.
image_np, input_tensor, output_dict = run_inference_for_single_image(model, image)

In [41]:
# Get relevant predictions and display.
preds = get_predictions(output_dict)
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    preds['detection_boxes'],
    preds['detection_classes'],
    preds['detection_scores'],
    category_index,
    instance_masks=preds.get('detection_masks_reframed', None),
    use_normalized_coordinates=True,
    line_thickness=1
)

display(Image.fromarray(image_np))

Output hidden; open in https://colab.research.google.com to view.

In [42]:
print(preds.keys())
preds['detection_classes']

dict_keys(['num_detections', 'detection_boxes', 'detection_classes', 'detection_masks', 'detection_scores', 'detection_masks_reframed'])


array([ 3, 10,  8, 64, 10, 10,  1, 64,  1, 10,  1, 10,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, 10,  1,  1, 10, 64, 10,  1,  1,  1,  1,  1,
       64,  1,  1,  1,  1,  1,  1,  1, 85,  1, 64,  1, 10,  1,  1,  1,  1,
        1,  3,  1, 31, 31, 64,  1, 10, 10, 10,  1,  1, 10,  1, 28, 86,  1,
        1,  1,  1,  1,  1,  1, 10,  1,  1, 42, 27, 31, 31, 10,  1,  1, 27,
        1, 31,  1,  1,  1,  1,  1,  1,  1, 15,  1,  8, 13,  1, 10])